In [1]:
import pandas as pd
import numpy as np
import csv
import os
import warnings
# from pandas.core.common import SettingWithCopyWarning

# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
HITTER_FILE = '../mlb_data/hittersByGame.csv'
PITCHER_FILE = '../mlb_data/pitchersByGame.csv'
GAME_FILE = '../mlb_data/games.csv'

In [2]:
# mapping between game ID and season
map_gameid_season = dict()
raw_data_game = pd.read_csv(GAME_FILE)
for i in range(len(raw_data_game)):
    map_gameid_season[raw_data_game['Game'][i]] = raw_data_game['Date'][i][0:4]

In [3]:
# Accumulate pitcher data over each season
pitcher_data = {'2016': dict(), '2017': dict(), '2018': dict(), '2019': dict(), '2020': dict(), '2021': dict()}
raw_data_pitcher = pd.read_csv(PITCHER_FILE)
for i in range(len(raw_data_pitcher)):
    pitcher = raw_data_pitcher['Pitchers'][i]
    if pitcher == 'TEAM':
        continue
    season = map_gameid_season[raw_data_pitcher['Game'][i]]
    team = raw_data_pitcher['Team'][i]
    if team not in pitcher_data[season]:
        pitcher_data[season][team] = dict()
    if pitcher not in pitcher_data[season][team]:
        pitcher_data[season][team][pitcher] = {'IP': 0, 'H': 0, 'R': 0, 'ER': 0, 'BB': 0, 'K': 0, 'HR': 0}
    for cat in pitcher_data[season][team][pitcher]:
        pitcher_data[season][team][pitcher][cat] += float(raw_data_pitcher[cat][i])

/var/folders/fv/7x5lxpms481dq9vtq7ngksqw0000gn/T/ipykernel_3949/3421357441.py:3: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_pitcher = pd.read_csv(PITCHER_FILE)


In [4]:
# Store the data into csv
csv_columns = ['Season', 'Teams', 'Pitcher', 'IP', 'H', 'R', 'ER', 'BB', 'K', 'HR', 'ERA']
dict_data = list()
for season in pitcher_data:
    for team in pitcher_data[season]:
        for pitcher in pitcher_data[season][team]:
            t = dict()
            t['Season'] = season
            t['Teams'] = team
            t['Pitcher'] = pitcher
            t['IP'] = int(pitcher_data[season][team][pitcher]['IP'])
            t['H'] = int(pitcher_data[season][team][pitcher]['H'])
            t['R'] = int(pitcher_data[season][team][pitcher]['R'])
            t['ER'] = int(pitcher_data[season][team][pitcher]['ER'])
            t['K'] = int(pitcher_data[season][team][pitcher]['K'])
            t['HR'] = int(pitcher_data[season][team][pitcher]['HR'])
            t['BB'] = int(pitcher_data[season][team][pitcher]['BB'])
            if t['IP'] != 0:
                t['ERA'] = "{0:.2f}".format(9 * t['ER'] / t['IP'])
            else:
                t['ERA'] = 0
            dict_data.append(t)
csv_file = "../data_processed/pitcher_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for d in dict_data:
            writer.writerow(d)
except IOError:
    print("I/O error")

In [5]:
# Accumulate pitcher data over each season
hitter_data = {'2016': dict(), '2017': dict(), '2018': dict(), '2019': dict(), '2020': dict(), '2021': dict()}
raw_data_hitter = pd.read_csv(HITTER_FILE)
for i in range(len(raw_data_hitter)):
    hitter = raw_data_hitter['Hitters'][i]
    if hitter == 'TEAM' or raw_data_hitter['H-AB'][i] == '-----':
        continue
    season = map_gameid_season[raw_data_hitter['Game'][i]]
    team = raw_data_hitter['Team'][i]
    if team not in hitter_data[season]:
        hitter_data[season][team] = dict()
    if hitter not in hitter_data[season][team]:
        hitter_data[season][team][hitter] = {'AB': 0, 'R': 0, 'H': 0, 'RBI': 0, 'BB': 0, 'K': 0, 'SLG': 0}
    for cat in hitter_data[season][team][hitter]:
        if cat == 'SLG':
            if raw_data_hitter['SLG'][i] != '--':
                hitter_data[season][team][hitter]['SLG'] += float(raw_data_hitter['SLG'][i]) * float(raw_data_hitter['AB'][i])
        else:
            hitter_data[season][team][hitter][cat] += float(raw_data_hitter[cat][i])

/var/folders/fv/7x5lxpms481dq9vtq7ngksqw0000gn/T/ipykernel_3949/918351986.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_hitter = pd.read_csv(HITTER_FILE)


In [6]:
# Store the data into csv
csv_columns_hitter = ['Season', 'Teams', 'Hitter', 'AB', 'R', 'H', 'RBI', 'BB', 'K', 'SLG', 'AVG', 'OBP']
dict_data_hitter = list()
for season in hitter_data:
    for team in hitter_data[season]:
        for hitter in hitter_data[season][team]:
            t = dict()
            t['Season'] = season
            t['Teams'] = team
            t['Hitter'] = hitter
            t['AB'] = int(hitter_data[season][team][hitter]['AB'])
            t['R'] = int(hitter_data[season][team][hitter]['R'])
            t['H'] = int(hitter_data[season][team][hitter]['H'])
            t['RBI'] = int(hitter_data[season][team][hitter]['RBI'])
            t['BB'] = int(hitter_data[season][team][hitter]['BB'])
            t['K'] = int(hitter_data[season][team][hitter]['K'])
            if hitter_data[season][team][hitter]['AB'] != 0:
                t['SLG'] = "{0:.3f}".format(hitter_data[season][team][hitter]['SLG'] / hitter_data[season][team][hitter]['AB'])
                t['AVG'] = "{0:.3f}".format(t['H'] / t['AB'])
                t['OBP'] = "{0:.3f}".format((t['H'] + t['R']) / (t['AB'] + t['R']))
            else:
                t['SLG'] = 0
                t['AVG'] = 0
                t['OBP'] = 0
            dict_data_hitter.append(t)
csv_file = "../data_processed/hitter_data.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns_hitter)
        writer.writeheader()
        for d in dict_data_hitter:
            writer.writerow(d)
except IOError:
    print("I/O error")

In [7]:
pitcher_season_best = {'2016': dict(), '2017': dict(), '2018': dict(), '2019': dict(), '2020': dict(), '2021': dict()}
dic = ['IP', 'H', 'R', 'ER', 'BB', 'K', 'HR', 'ERA']
for season in pitcher_data:
    for team in pitcher_data[season]:
        for pitcher in pitcher_data[season][team]:
            inning = pitcher_data[season][team][pitcher]['IP']
            if inning < 50:
                continue
            for d in dic:
                if d not in pitcher_season_best[season]:
                    pitcher_season_best[season][d] = list()
                if d != 'ERA':
                    if d == 'IP':
                        pitcher_season_best[season][d].append({'name': pitcher, 'info': [team, pitcher_data[season][team][pitcher][d]]})
                    else:
                        pitcher_season_best[season][d].append({'name': pitcher, 'info': [team, pitcher_data[season][team][pitcher][d]/inning]})
                else:
                    pitcher_season_best[season]['ERA'].append({'name': pitcher, 'info': [team, 9 * pitcher_data[season][team][pitcher]['ER'] / pitcher_data[season][team][pitcher]['IP']]})

In [8]:
for year in pitcher_season_best:
    for ind in pitcher_season_best[year]:
        if ind == 'IP' or ind == 'K':
            pitcher_season_best[year][ind] = sorted(pitcher_season_best[year][ind], key=lambda d: d['info'][1], reverse = True)
        else:
            pitcher_season_best[year][ind] = sorted(pitcher_season_best[year][ind], key=lambda d: d['info'][1])
        pitcher_season_best[year][ind] = pitcher_season_best[year][ind][0:10]

In [9]:
csv_columns_pitcher_top10 = ['Season', 'Team', 'Pitcher', 'Indicator', 'Value']
dict_data_pitcher_top10 = list()
csv_file = "../data_processed/pitcher_top10.csv"
for season in pitcher_season_best:
    for cat in pitcher_season_best[season]:
        for i in range(10):
            t = dict()
            t['Season'] = season
            t['Team'] = pitcher_season_best[season][cat][i]['info'][0]
            t['Value'] = "{0:.2f}".format(pitcher_season_best[season][cat][i]['info'][1])
            t['Pitcher'] = pitcher_season_best[season][cat][i]['name']
            t['Indicator'] = cat
            dict_data_pitcher_top10.append(t)
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns_pitcher_top10)
        writer.writeheader()
        for d in dict_data_pitcher_top10:
            writer.writerow(d)
except IOError:
    print("I/O error")

In [10]:
hitter_season_best = {'2016': dict(), '2017': dict(), '2018': dict(), '2019': dict(), '2020': dict(), '2021': dict()}
dic = ['AB', 'R', 'H', 'RBI', 'BB', 'K', 'SLG', 'AVG', 'OBP']
for season in hitter_data:
    for team in hitter_data[season]:
        for hitter in hitter_data[season][team]:
            ab = hitter_data[season][team][hitter]['AB']
            if ab < 254 and season == '2020':
                continue
            if ab < 502 and season != '2020':
                continue
            for d in dic:
                if d not in hitter_season_best[season]:
                    hitter_season_best[season][d] = list()
                if d == 'AB':
                    hitter_season_best[season][d].append({'name': hitter, 'info': [team, hitter_data[season][team][hitter][d]]})
                elif d == 'SLG':
                    hitter_season_best[season][d].append({'name': hitter, 'info': [team, hitter_data[season][team][hitter]['SLG'] / hitter_data[season][team][hitter]['AB']]})
                elif d == 'AVG':
                    hitter_season_best[season][d].append({'name': hitter, 'info': [team, hitter_data[season][team][hitter]['H'] / ab]})
                elif d == 'OBP':
                    hitter_season_best[season][d].append({'name': hitter, 'info': [team, (hitter_data[season][team][hitter]['H'] + hitter_data[season][team][hitter]['R']) / (ab + hitter_data[season][team][hitter]['R'])]})
                else:
                    hitter_season_best[season][d].append({'name': hitter, 'info': [team, hitter_data[season][team][hitter][d]/ab]})


In [11]:
for year in hitter_season_best:
    for ind in hitter_season_best[year]:
        if ind != 'K':
            hitter_season_best[year][ind] = sorted(hitter_season_best[year][ind], key=lambda d: d['info'][1], reverse = True)
        else:
            hitter_season_best[year][ind] = sorted(hitter_season_best[year][ind], key=lambda d: d['info'][1])
        hitter_season_best[year][ind] = hitter_season_best[year][ind][0:10]

In [12]:
csv_columns_hitter_top10 = ['Season', 'Team', 'Hitter', 'Indicator', 'Value']
dict_data_hitter_top10 = list()
csv_file = "../data_processed/hitter_top10.csv"
for season in hitter_season_best:
    for cat in hitter_season_best[season]:
        for i in range(10):
            t = dict()
            t['Season'] = season
            t['Team'] = hitter_season_best[season][cat][i]['info'][0]
            t['Value'] = "{0:.3f}".format(hitter_season_best[season][cat][i]['info'][1])
            t['Hitter'] = hitter_season_best[season][cat][i]['name']
            t['Indicator'] = cat
            dict_data_hitter_top10.append(t)
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns_hitter_top10)
        writer.writeheader()
        for d in dict_data_hitter_top10:
            writer.writerow(d)
except IOError:
    print("I/O error")